SE network pytorch implementation from Start


In [1]:
# # Imports
import argparse
import collections
import math
import time

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import metrics, preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

import geniter1_disjoint
import record
import Utils1

#import torch_optimizer as optim2   """torch_optimizer is not present in the Env"""

from torchsummary import summary
from vit_pytorch_ours import ViT

In [2]:
def sampling(proportion, ground_truth):
    train = {}
    test = {}
    labels_loc = {}
    m = max(ground_truth)
    for i in range(m):
        indexes = [
            j for j, x in enumerate(ground_truth.ravel().tolist())
            if x == i + 1
        ]
        np.random.shuffle(indexes)
        labels_loc[i] = indexes
        if proportion != 1:
            nb_val = max(int((1 - proportion) * len(indexes)), 3)
        else:
            nb_val = 0
        train[i] = indexes[:nb_val]
        test[i] = indexes[nb_val:]
    train_indexes = []
    test_indexes = []
    for i in range(m):
        train_indexes += train[i]
        test_indexes += test[i]
    np.random.shuffle(train_indexes)
    np.random.shuffle(test_indexes)
    return train_indexes, test_indexes



def shuffle_data(data_hsi, gt_hsi):
    data_hsi_shuffle = np.zeros((data_hsi.shape[0], data_hsi.shape[1], data_hsi.shape[2], data_hsi.shape[3]))
    gt_hsi_shuffle = np.zeros(data_hsi.shape[0])
    _, total_indexes = sampling(1, gt_hsi)
    for i in range(len(total_indexes)):
        data_hsi_shuffle[i] = data_hsi[total_indexes[i],:,:,:]
        gt_hsi_shuffle[i] = gt_hsi[total_indexes[i]] 
    gt_hsi_shuffle = gt_hsi_shuffle.astype(int) 
    return data_hsi_shuffle, gt_hsi_shuffle


In [ ]:
PARAM_DATASET = 'IN'
PARAM_EPOCH = 70
PARAM_ITER = 3    # number of iterations the model is trained and tested
PATCH_SIZE = 2     # HSI patch/window size = 2*PATCH_SIZE +1
PARAM_VAL = 0.2    
PARAM_OPTIM = 'adam'
batch_size = 32


# Data Loading


In [ ]:
def load_dataset(Dataset, split=0.7):
    data_path = '/home/sankar/Crop_dataset/data/'     # data path
    data_hsi = np.load(data_path + 'X_train_10Crops_patch11_.npy')   # reading the data
    gt_hsi = np.load(data_path + 'y_train_10Crops_patch11_.npy')   # reading labels 
  
    gt_hsi = gt_hsi.astype(int) 
    data_hsi, gt_hsi = shuffle_data(data_hsi, gt_hsi)
    
    K = data_hsi.shape[2]
    TOTAL_SIZE = np.count_nonzero(np.asarray(gt_hsi))
    
    VALIDATION_SPLIT = split
    TRAIN_SIZE = math.ceil(TOTAL_SIZE * VALIDATION_SPLIT)
   
    
    return data_hsi, gt_hsi, TOTAL_SIZE, TRAIN_SIZE, VALIDATION_SPLIT

In [5]:
# # Data Loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# for Monte Carlo runs
seeds = [1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341]
ensemble = 1

global Dataset  # UP,IN,SV, KSC
dataset = PARAM_DATASET  #input('Please input the name of Dataset(IN, UP, SV, KSC):')
Dataset = dataset.upper()


In [6]:
# # Pytorch Data Loader Creation
data_hsi, gt_hsi, TOTAL_SIZE, TRAIN_SIZE, VALIDATION_SPLIT = load_dataset(
    Dataset, PARAM_VAL)
print(data_hsi.shape)
tot_samples, image_x, image_y, BAND = data_hsi.shape

#data = data_hsi
data = data_hsi.reshape(
np.prod(data_hsi.shape[:3]), np.prod(data_hsi.shape[3:]))
print(data.shape)
gt = gt_hsi 

CLASSES_NUM = max(gt_hsi)
#CLASSES_NUM = CLASSES_NUM. astype(int)
print('The class numbers of the HSI data is:', CLASSES_NUM)

(2176, 25, 25, 220)
(1360000, 220)
The class numbers of the HSI data is: 4


In [12]:
print('-----Importing Setting Parameters-----')
ITER = PARAM_ITER
PATCH_LENGTH = PATCH_SIZE
lr, num_epochs, batch_size = 0.001, 11, batch_size
loss = torch.nn.CrossEntropyLoss()

img_rows = 2 * PATCH_LENGTH + 1
img_cols = 2 * PATCH_LENGTH + 1
img_channels = data_hsi.shape[3]
INPUT_DIMENSION = data_hsi.shape[3]
ALL_SIZE = data_hsi.shape[0] 

VAL_SIZE = TOTAL_SIZE - TRAIN_SIZE
print(VAL_SIZE)
ALL_SIZE, TRAIN_SIZE, VAL_SIZE

-----Importing Setting Parameters-----
1740


(2176, 436, 1740)

In [13]:
KAPPA = []
OA = []
AA = []
TRAINING_TIME = []
TESTING_TIME = []
ELEMENT_ACC = np.zeros((ITER, CLASSES_NUM))
ELEMENT_ACC.shape
data.shape
data = preprocessing.scale(data)

data_ = data.reshape(data_hsi.shape[0], data_hsi.shape[1], data_hsi.shape[2], data_hsi.shape[3] )
print(data_.shape)
whole_data = data_

(2176, 25, 25, 220)


# Reading Test Data Samples

In [ ]:
def load_dataset_test(Dataset, split=0.99):
    data_path = '/home/sankar/Crop_dataset/data/'   
    data_hsi = np.load(data_path + 'X_test_10Crops_patch11_.npy')   
    data_hsi = data_hsi[:, 2:7, 2:7, :]
    gt_hsi = np.load(data_path + 'y_test_10Crops_patch11_.npy')   

    gt_hsi = gt_hsi.astype(int) 
    data_hsi, gt_hsi = shuffle_data(data_hsi, gt_hsi)
    
    K = data_hsi.shape[2]
    TOTAL_SIZE = np.count_nonzero(np.asarray(gt_hsi))
    
    VALIDATION_SPLIT = split
    TRAIN_SIZE = math.ceil(TOTAL_SIZE * VALIDATION_SPLIT)
   
    
    return data_hsi, gt_hsi, TOTAL_SIZE, TRAIN_SIZE, VALIDATION_SPLIT

# # Pytorch Data Loader Creation
data_hsi_disj, gt_hsi_disj, _, _, _ = load_dataset_test(Dataset, 0.99)
print(data_hsi_disj.shape)
data_disj = data_hsi_disj.reshape(np.prod(data_hsi_disj.shape[:3]), np.prod(data_hsi_disj.shape[3:]))
data_disj = preprocessing.scale(data_disj)
whole_data_disj = data_disj.reshape(data_hsi_disj.shape[0], data_hsi_disj.shape[1], data_hsi_disj.shape[2], data_hsi_disj.shape[3] )


In [14]:
def train(net,
          train_iter,
          valida_iter,
          loss,
          optimizer,
          device,
          epochs,
          early_stopping=True,
          early_num=20):
    loss_list = [100]
    early_epoch = 0

    net = net.to(device)
    print("training on ", device)
    start = time.time()
    train_loss_list = []
    valida_loss_list = []
    train_acc_list = []
    valida_acc_list = []
    for epoch in range(epochs):
        train_acc_sum, n = 0.0, 0
        time_epoch = time.time()
        lr_adjust = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, 15, eta_min=0.0, last_epoch=-1)
        for X, y in train_iter:

            batch_count, train_l_sum = 0, 0
            #X = X.permute(0, 3, 1, 2)
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            # print('y_hat', y_hat)
            # print('y', y)
            l = loss(y_hat, y.long())

            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        lr_adjust.step()
        valida_acc, valida_loss = record.evaluate_accuracy(
            valida_iter, net, loss, device)
        loss_list.append(valida_loss)

        train_loss_list.append(train_l_sum)  # / batch_count)
        train_acc_list.append(train_acc_sum / n)
        valida_loss_list.append(valida_loss)
        valida_acc_list.append(valida_acc)

        print(
            'epoch %d, train loss %.6f, train acc %.3f, valida loss %.6f, valida acc %.3f, time %.1f sec'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               valida_loss, valida_acc, time.time() - time_epoch))

        PATH = "./3DCNN_ViT.pt"
        # if loss_list[-1] <= 0.01 and valida_acc >= 0.95:
        #     torch.save(net.state_dict(), PATH)
        #     break

        if early_stopping and loss_list[-2] < loss_list[-1]:
            if early_epoch == 0:  # and valida_acc > 0.9:
                torch.save(net.state_dict(), PATH)
            early_epoch += 1
            loss_list[-1] = loss_list[-2]
            if early_epoch == early_num:
                net.load_state_dict(torch.load(PATH))
                break
        else:
            early_epoch = 0

    print('epoch %d, loss %.4f, train acc %.3f, time %.1f sec'
          % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
             time.time() - start))

In [ ]:

mode = 'ViT' #'ViT'   'CAF'
num_classes = 10
depth = 5
near_band = 1
image_size = 5  # patch or window size
batch_size = 32
num_patches = 300 # number of channels in the HSI
PARAM_EPOCH = 65

from vit_pytorch_ours import ViT

In [ ]:

# # Training
for index_iter in range(ITER):
    print('iter:', index_iter)
    # define the model
    net = model = ViT(image_size = image_size, near_band = near_band, num_patches = num_patches, num_classes = num_classes, dim = 64, depth = depth, heads = 4, mlp_dim = 8, dropout = 0.1, emb_dropout = 0.1, mode = mode )

    if PARAM_OPTIM == 'diffgrad':
        optimizer = optim2.DiffGrad(
            net.parameters(),
            lr=lr,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0)  # weight_decay=0.0001)
    if PARAM_OPTIM == 'adam':
        optimizer = optim.Adam(
            net.parameters(),
            lr=1e-3,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0)
    time_1 = int(time.time())
    np.random.seed(seeds[index_iter])
    # train_indices, test_indices = select(gt)
    train_indices, test_indices = sampling(VALIDATION_SPLIT, gt)
    _, total_indices = sampling(1, gt)

    TRAIN_SIZE = len(train_indices)
    print('Train size: ', TRAIN_SIZE)
    TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE
    VAL_SIZE = TEST_SIZE
    print('Validation size: ', VAL_SIZE)

    print('-----Selecting Small Pieces from the Original Cube Data-----')
    train_iter, test_iter, all_iter = geniter1_disjoint.generate_iter(
        TRAIN_SIZE, train_indices, TEST_SIZE, test_indices, TOTAL_SIZE,
        total_indices, whole_data, PATCH_LENGTH, INPUT_DIMENSION, batch_size, gt) #batchsize in 1
    valida_iter = test_iter



    _, test_indices_disj = sampling(0.99, gt_hsi_disj)
    TEST_SIZE_disj = len(test_indices_disj)
    test_iter_disj = geniter1_disjoint.generate_iter_test(TEST_SIZE_disj, test_indices_disj, whole_data_disj, PATCH_LENGTH, INPUT_DIMENSION, batch_size, gt_hsi_disj) #batchsize in 1

    tic1 = time.time()
    train(
        net,
        train_iter,
        valida_iter,
        loss,
        optimizer,
        device,
        epochs=PARAM_EPOCH)
    toc1 = time.time()

    pred_test = []
    y_test= []
    tic2 = time.time()
    with torch.no_grad():
        for X, y in test_iter_disj:
            X = X.to(device)
            net.eval()
            y_hat = net(X)
            pred_test.extend(np.array(net(X).cpu().argmax(axis=1)))
            y_test.extend(np.array(y))
    toc2 = time.time()
    collections.Counter(pred_test)
    
    gt_test = y_test  # gt_hsi_disj[test_indices_disj] - 1  
    overall_acc = metrics.accuracy_score(gt_test, pred_test)
    confusion_matrix = metrics.confusion_matrix(gt_test, pred_test)
    each_acc, average_acc = record.aa_and_each_accuracy(confusion_matrix)
    kappa = metrics.cohen_kappa_score(gt_test, pred_test)
    
    torch.save(net.state_dict(), '3DCNN_ViT_Net' + str(round(overall_acc, 3)) + '.pt')
    KAPPA.append(kappa)
    OA.append(overall_acc)
    AA.append(average_acc)
    TRAINING_TIME.append(toc1 - tic1)
    TESTING_TIME.append(toc2 - tic2)
    ELEMENT_ACC[index_iter, :] = each_acc

iter: 0
Train size:  7943
Validation size:  1988
-----Selecting Small Pieces from the Original Cube Data-----
(7943, 5, 5, 300) (7943,)
Test data shape:
(8330, 5, 5, 300) (8330,)
training on  cuda
epoch 1, train loss 0.278873, train acc 0.622, valida loss 0.185540, valida acc 0.830, time 6.4 sec
epoch 2, train loss 0.319461, train acc 0.844, valida loss 0.619777, valida acc 0.900, time 6.7 sec
epoch 3, train loss 0.167643, train acc 0.897, valida loss 0.294037, valida acc 0.933, time 6.5 sec
epoch 4, train loss 0.090859, train acc 0.931, valida loss 0.635709, valida acc 0.958, time 6.5 sec
epoch 5, train loss 0.033730, train acc 0.944, valida loss 0.533309, valida acc 0.971, time 6.5 sec
epoch 6, train loss 0.207132, train acc 0.959, valida loss 0.853393, valida acc 0.957, time 6.6 sec


KeyboardInterrupt: 

In [35]:
######Groundnut with ViT+3D CNN
print(OA, AA, KAPPA, ELEMENT_ACC) #, confusion_matrix
print("On PaviaU Data set")
print('OA is:' + str(np.mean(OA)) + '+/-'+ str(np.std(OA))),
print('AA is:' + str(np.mean(AA)) + '+/-'+ str(np.std(AA))),
print('KAPPA is:' + str(np.mean(KAPPA)) + '+/-'+ str(np.std(KAPPA))),
# print('Train time:' + str(np.mean(TRAINING_TIME)) + '+/-'+ str(np.std(TRAINING_TIME))),
# print('TESTING_TIME time:' + str(np.mean(TESTING_TIME)) + '+/-'+ str(np.std(TESTING_TIME))),

[0.9787515006002401, 0.955702280912365, 0.9900360144057623, 0.99015606242497, 0.9991596638655462, 0.9926770708283313] [0.9869938883396692, 0.9732195638662994, 0.9933460834769082, 0.9936541805345698, 0.9989880698778449, 0.9950823306216297] [0.9718393122788036, 0.9417227958519832, 0.9867537262533065, 0.986917783576902, 0.9988807687412357, 0.9902609502976503] [[1.         0.9421572  0.99793388 1.         0.99487836]
 [0.99867725 0.88159238 0.98966942 1.         0.99615877]
 [0.9973545  0.97754338 0.99311295 1.         0.99871959]
 [0.99955908 0.97550187 0.99449036 1.         0.99871959]
 [0.99955908 0.9993195  0.99862259 1.         0.99743918]
 [1.         0.98298741 0.99242424 1.         1.        ]]
On PaviaU Data set
OA is:0.9844137655062024+/-0.014181666458907251
AA is:0.9902140194528202+/-0.008381623876917833
KAPPA is:0.9793958894999802+/-0.018640415596613204


(None,)